In [ ]:
import matplotlib
import sys
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import time

start_time=time.time()

T = np.arange(1, 3.1, 0.1);
N=int(input("Please, type the length N of the NxN spin array used to generate data:"))
Algorithm=input("Type k for Kawasaki or g for Glauber dynamics:" "")

#If the algorithm is Glauber:
if Algorithm == "g": 
    
    #Open data files
    EnergyDat = pd.read_csv("fenergyGlauberN50.csv")
    MagnetDat = pd.read_csv("fmagnetGlauberN50.csv")

    
    #Compute averages from energy and magnetization data
    Eplot= EnergyDat.mean(axis=1)
    Mplot= MagnetDat.mean(axis=1)
    E2=np.average(EnergyDat**2,axis=1)
    M2=np.average(MagnetDat**2,axis=1)

    #Compute susceptibility and heat capacity from energy and magnetization data
    C = (E2 - (Eplot**2) )/((T**2)*(N**2))
    X = (M2 - (Mplot**2))/(T*(N**2))
  
  
    #Calculation of heat capacity error

    stdC=np.zeros(len(T))
    Cshuffle=np.zeros(len(T))
    C2shuffle=np.zeros(len(T))
    
    for t in range(len(T)):
        
        shuffles=np.zeros((100,EnergyDat.loc[[1]].size))
        
        Cshuffle=np.zeros(100)
        C2shuffle=np.zeros(100)
        Eshuffmean=np.zeros(100)
        E2shuffmean=np.zeros(100)
        
        for i in range(100):
            for j in range(EnergyDat.loc[[1]].size):
                
                position=int(np.random.rand()*EnergyDat.loc[[1]].size)
                shuffles[i,j] = EnergyDat.iat[t,position]
            
            Eshuffmean[i]=np.average(shuffles[i])
            E2shuffmean[i]=np.average(shuffles[i]**2)
            Cshuffle[i] = (E2shuffmean[i] - (Eshuffmean[i]**2) )/((T[t]**2)*(N**2)) 
            C2shuffle[i] = Cshuffle[i]**2
        
        stdC[t] = math.sqrt(np.average(C2shuffle)-np.average(Cshuffle)**2)
    
    #Create data files of ploted data points
    energyPlotPoints=pd.DataFrame(Eplot)
    energyPlotPoints.to_csv('energyPlotGlauberN'+str(N)+'.csv')
    CPlotPoints=pd.DataFrame(C)
    CPlotPoints.to_csv('CPlotGlauberN'+str(N)+'.csv')
    magnetPlotPoints=pd.DataFrame(abs(Mplot))
    magnetPlotPoints.to_csv('magnetPlotGlauberN'+str(N)+'.csv')
    XPlotPoints=pd.DataFrame(X)
    XPlotPoints.to_csv('XPlotGlauberN'+str(N)+'.csv')
    
    #Plot energy
    f = plt.figure(figsize=(18, 10));    
    plt.scatter(T, Eplot, s=50, marker='o', color='IndianRed')
    plt.title("Energy vs Temperature with an array of " + str(N*N) + " spins and "+ str(EnergyDat.loc[[1]].size*10)+ " Montecarlo steps (Glauber)")
    plt.xlabel("Temperature (T)", fontsize=20);
    plt.ylabel("Energy ", fontsize=20);         plt.axis('tight');

    #Plot Magnetization
    g = plt.figure(figsize=(18, 10));
    plt.scatter(T, abs(Mplot), s=50, marker='o', color='RoyalBlue')
    plt.title("Magnetization vs Temperature with an array of " + str(N*N) + " spins and "+ str(EnergyDat.loc[[1]].size*10)+ " Montecarlo steps (Glauber)")
    plt.xlabel("Temperature", fontsize=20); 
    plt.ylabel("Magnetization ", fontsize=20);   plt.axis('tight');

    #Plot heat capacity
    h = plt.figure(figsize=(18, 10));
    plt.scatter(T, C, s=50, marker='o', color='IndianRed')
    #plt.errorbar(T,C,yerr=stdC)
    plt.title("Heat Capacity vs Temperature with an array of " + str(N*N) + " spins and "+ str(EnergyDat.loc[[1]].size*10)+ " Montecarlo steps (Glauber)")
    plt.xlabel("Temperature", fontsize=20);  
    plt.ylabel("Specific Heat ", fontsize=20);   plt.axis('tight');   

    #Plot susceptibility
    d = plt.figure(figsize=(18, 10));
    plt.scatter(T, X, s=50, marker='o', color='RoyalBlue')
    plt.title("Susceptibility vs Temperature with an array of " + str(N*N) + " spins and "+ str(EnergyDat.loc[[1]].size*10)+ " Montecarlo steps and Glauber dynamics" )
    plt.xlabel("Temperature", fontsize=20); 
    plt.ylabel("Susceptibility", fontsize=20);   plt.axis('tight');

#If the algorithm is Kawasaki:        
elif Algorithm == "k":
 
    #Open data files
    EnergyDat = pd.read_csv("fenergyKawasakiN50.csv")
    Cdata= pd.read_csv("fCKawasakiN50.csv")
    
    #Compute average of extrated energy data
    Eplot= EnergyDat.mean(axis=1)
    E2=np.average(EnergyDat**2,axis=1)

    #Compute heat capacity using energy data
    C = (E2 - (Eplot**2) )/((T**2)*(N**2))
    
    #Calculation of heat capacity error (bootrstrap resampling)
    stdC=np.zeros(len(T))
    Cshuffle=np.zeros(len(T))
    C2shuffle=np.zeros(len(T))
    
    #Create data files of ploted data points
    energyPlotPoints=pd.DataFrame(Eplot)
    energyPlotPoints.to_csv('energyPlotKawasakiN'+str(N)+'.csv')
    CPlotPoints=pd.DataFrame(C)
    CPlotPoints.to_csv('CPlotKawasakiN'+str(N)+'.csv')
    for t in range(len(T)):
        
        shuffles=np.zeros((100,EnergyDat.loc[[1]].size))
        
        Cshuffle=np.zeros(100)
        C2shuffle=np.zeros(100)
        Eshuffmean=np.zeros(100)
        E2shuffmean=np.zeros(100)
        
        for i in range(100):
            for j in range(EnergyDat.loc[[1]].size):
                
                position=int(np.random.rand()*EnergyDat.loc[[1]].size)
                shuffles[i,j] = EnergyDat.iat[t,position]
            
            Eshuffmean[i]=np.average(shuffles[i])
            E2shuffmean[i]=np.average(shuffles[i]**2)
            Cshuffle[i] = (E2shuffmean[i] - (Eshuffmean[i]**2) )/((T[t]**2)*(N**2)) 
            C2shuffle[i] = Cshuffle[i]**2
        
        stdC[t] = math.sqrt(np.average(C2shuffle)-np.average(Cshuffle)**2)
    
    
    #Plot energy
    f = plt.figure(figsize=(18, 10));   
    plt.scatter(T, Eplot, s=50, marker='o', color='IndianRed')
    plt.title("Energy vs Temperature with an array of " + str(N*N) + " spins and "+ str(EnergyDat.loc[[1]].size*10)+ " Montecarlo steps (Kawasaki)")
    plt.xlabel("Temperature", fontsize=20);
    plt.ylabel("Energy", fontsize=20);         plt.axis('tight');
    
    #Plot heat capacity
    g=plt.figure(figsize=(18, 10));
    plt.scatter(T, C, s=50, marker='o', color='IndianRed')
    plt.errorbar(T,C,yerr=stdC)
    plt.title("Heat Capacity vs Temperature with an array of " + str(N*N) + " spins and "+ str(EnergyDat.loc[[1]].size*10)+ " Montecarlo steps (Kawasaki)")
    plt.xlabel("Temperature", fontsize=20);  
    plt.ylabel("Specific Heat", fontsize=20);   plt.axis('tight');   

        
else:
    
    print("You did not press k nor g, please try again.")

print("It took: "+str((time.time()-start_time)/60)+" minutes to run")